In [1]:
# import numpy 
import numpy as np 

# import pandas
import pandas as pd 

# import regex
import re

# import nltk
import nltk 

In [2]:
# load data
data_source_url = "https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv"
airline_tweets = pd.read_csv(data_source_url)

* print top 5 rows

In [3]:
airline_tweets.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


* use 'text' column to create array with name 'features'
* use 'airline_sentiment' column to create array with name 'labels'

In [ ]:
features = airline_tweets['text']
labels = airline_tweets['airline_sentiment']

* clean the text in features array
* Remove all the special characters
* remove all single characters
* Remove single characters from the start
* Substituting multiple spaces with single space
* Removing prefixed 'b'
* Converting to Lowercase

In [ ]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    sentence = re.sub(r'''[\*\~]+''', "", sentence)

    # remove all single characters
    sentence = re.sub(r"\b[a-zA-Z]\b", "", sentence)

    # Remove single characters from the start
    

    # Substituting multiple spaces with single space
    sentence = re.sub(r'\s+', ' ', sentence)
    # Removing prefixed 'b'
    

    # Converting to Lowercase
    sentence = sentence.lower()
    processed_features.append(sentence)
    
def normalize_text(documents, 
                   min_token_len=2, 
                   irrelevant_pos=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    documents -- (np.array[str]) the list of documents to be preprocessed
    min_token_len -- (int) min_token_length required
    irrelevant_pos -- (list) a list of irrelevant pos tags
    
    Returns: np.array[str] the normalized documents
    """
    normalized_documents = []
    
    for text in documents:
        # Remove Emails
        text = re.sub(r'\S*@\S*\s?', '', text)

        # Remove extra space characters
        text = re.sub(r'\s+', ' ', text)

        # Remove distracting characters
        text = re.sub(r'''[\*\~]+''', "", text)

        doc = nlp(text)
        clean_text = []

        for token in doc:
            if (token.is_stop == False # Check if it's not a stopword
                and token.is_alpha # Check if it's an alphanumerics char
                and len(token) > min_token_len # Check if the word meets minimum threshold
                and token.pos_ not in irrelevant_pos): # Check if the POS is in the acceptable POS tags
                lemma = token.lemma_ # Take the lemma of the word
                clean_text.append(lemma)
                
        clean_text = ' '.join(clean_text)
        normalized_documents.append(clean_text)
        
    normalized_documents = np.array(normalized_documents)
    return normalized_documents    

* import stopwords from nltk

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))
stopw = stopwords.words('english')

* import TfidfVectorizer from sklearn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

* create vectorizer inastance of TfidfVectorizer with following settings:
* max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english')

In [ ]:
tfvec = TfidfVectorizer(max_features-2500,min_df=7,max_df=0.8,stopwords=stopw)

* transform features with vectorizer

In [ ]:
processed_features = tfvec.fit_transform(processed_features)

* import train_test_split from sklearn and split the data

In [ ]:
from sklearn.model_selection import trai_test_split
x_train, x_test, y_train, y_test = train_test_split(processed_features,labels,train_size=0.7,random_state=123)

* import classifier of your choice from sklearn (e.g. Random Forest, LogReg, Naive Bayes)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

* fit your classifier to data

In [ ]:
lr.fit(x_train,y_train)

* predict X_test

In [ ]:
y_pred = lr.predict(x_test)

* import confusion matrix and accuracy_score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

* print confusion matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

* print accaccuracy_score

In [ ]:
acc = accuracy_score(y_test,y_pred)
print(acc)
print(classification_report(y_test,y_pred))